# Importing libs

In [1]:
from datetime import datetime, timedelta

import numpy as np
import pandas as pd
import plotly.express as px
import yfinance as yf

from athena.src.services.backtest.backtest import Backtest
from athena.src.services.download.data import Data

# When is the start of the backtest?

In [2]:
start_date = datetime(2019, 12, 1)

# Downloading the data *takes ~25 seconds

In [ ]:
rules = {
    "domains": ["stocks", "variable etfs", "fixed etfs"]
}

download = Data(rules = rules, start_date = start_date - timedelta(2*365))
data = download.get_data(yahoo = False)
data

In [3]:
data = pd.read_csv("new_data", header = [0, 1], index_col = [0])
data.index = pd.to_datetime(data.index)

# Backtesting *takes ~50 seconds

In [4]:
def backtest(assets: int, fix_percentage: float, data: pd.DataFrame, rebalance: str, volume_days: int) -> pd.DataFrame:
    filters = {

        "max_price": {
            "cutoff": 70.0
        },

        "volume": {
            "num_tickers": 50,
            "days": volume_days
        },

        "momentum": {
            "num_tickers": assets,
            "months": 1
        }

    }
    
    if fix_percentage == 0:
        constraints = None

    else:
        constraints = {
            "invest_sections": {fix_percentage: ["FIXA11", "IRFM11"]}

        }

    optimizer = {
        "hrp": {
            "covariance_function": "exp_cov",
            "linkage_method": "average"
        }
    }

    backtest = Backtest(filters = filters, optimizer = optimizer, start_date = start_date, rebalance_interval = rebalance, data = data, constraints = constraints)
    backtest.backtest(plot_snapshot = False)
    
    return backtest.returns

In [14]:
# cons = backtest(volume_days = 20, assets = 6, fix_percentage = 0.75, data = data, rebalance = "monthly")
# mode = backtest(volume_days = 20, assets = 6, fix_percentage = 0.5, data = data, rebalance = "monthly")
# agre = backtest(volume_days = 20, assets = 6, fix_percentage = 0.25, data = data, rebalance = "monthly")
# vari = backtest(volume_days = 20, assets = 8, fix_percentage = 0, data = data, rebalance = "biweekly")
dyna = backtest(volume_days = 20, assets = 8, fix_percentage = (0, 0.8), data = data, rebalance = "biweekly")

# Analysing results

## Concateneting backtests

In [15]:
index   = cons.index
returns = pd.DataFrame(columns = ["Conservador", "Moderado", "Agressivo", "Variavel", "Dynamic", "IBOV"], index = index)

returns["Conservador"] = np.cumprod(1 + cons.values) - 1
returns["Moderado"]    = np.cumprod(1 + mode.values) - 1
returns["Agressivo"]   = np.cumprod(1 + agre.values) - 1
returns["Variavel"]    = np.cumprod(1 + vari[index]) - 1
returns["Dynamic"]     = np.cumprod(1 + dyna[index]) - 1

## Getting benchmark data

In [16]:
ibov = yf.download("^BVSP", start = start_date - timedelta(15))["Close"]

ibov_returns = ibov.pct_change()
same_dates = [date for date in index if date in ibov_returns.index]
ibov_returns = ibov_returns[same_dates]

for date in returns.index:
    if date not in ibov_returns.index:
        returns = returns.drop(index = date)

returns["IBOV"] = np.cumprod(1 + ibov_returns.values) - 1
returns = returns.astype(float)

[*********************100%***********************]  1 of 1 completed


# Plotting results

In [17]:
px.line(returns)

# Do you want to investigate a particular period?

In [ ]:
start = datetime(2021, 10, 1)
end   = datetime(2022, 3, 30)

# Plotting Results

In [ ]:
from datetime import datetime

dates = (ibov_returns.index > start) & (ibov_returns.index < end)
dates = ibov_returns.loc[dates].index

period_returns = pd.DataFrame(columns = ["Conservador", "Moderado", "Agressivo", "Variavel", "Dynamic", "IBOV"], index = dates)

period_returns["Conservador"] = np.cumprod(1 + cons.loc[dates]) - 1
period_returns["Moderado"]    = np.cumprod(1 + mode.loc[dates]) - 1
period_returns["Agressivo"]   = np.cumprod(1 + agre.loc[dates]) - 1
period_returns["Variavel"]    = np.cumprod(1 + vari.loc[dates]) - 1
period_returns["Dynamic"]     = np.cumprod(1 + dyna.loc[dates]) - 1
period_returns["IBOV"]        = np.cumprod(1 + ibov_returns[dates]) - 1

period_returns = period_returns.astype(float)

px.line(period_returns)